In [16]:
import numpy as np
import timeit
import numexpr as ne
from numba import njit

In [2]:
precession = np.random.uniform(1,10,[990, 1, 10, 3, 3])
vecMblood = np.random.uniform(1,10,[990, 1, 10, 3, 1])


In [3]:
# Time np.matmul
start = timeit.default_timer()
for _ in range(4000):
    np.matmul(precession, vecMblood)
end = timeit.default_timer()
print(f"np.matmul took {end - start} seconds.")

np.matmul took 0.6702912079999805 seconds.


In [4]:
# Time np.matmul
start = timeit.default_timer()
for _ in range(4000):
    np.einsum("...ij,...j", precession, vecMblood[..., 0])[..., None]
end = timeit.default_timer()
print(f"np.matmul took {end - start} seconds.")



np.matmul took 0.46126041599927703 seconds.


In [26]:
a =  np.einsum("...ij,...j", precession, vecMblood[..., 0])[..., None]

In [6]:
matmul_result = np.matmul(precession, vecMblood)
difference = np.abs(a - matmul_result)
print("Maximum difference:", np.max(difference))

Maximum difference: 2.842170943040401e-14


In [28]:
print(precession.shape)
print(vecMblood.shape)

(990, 1, 10, 3, 3)
(990, 1, 10, 3, 1)


In [7]:
rotX = np.random.uniform(1,10, [10, 3, 3])

a =  np.einsum("...ij,...j", rotX, vecMblood[..., 0])[..., None]

In [8]:
thetaZ = 5
rotX = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
rotY = np.array([[np.cos(thetaZ), -np.sin(thetaZ), 0],\
                    [np.sin(thetaZ), np.cos(thetaZ), 0],\
                    [0, 0, 1]])

In [9]:
a = np.matmul(rotY,rotX)
b = np.matmul(rotY,rotX)

b == rotY

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [10]:
rotY = np.zeros([len(thetaX),3,3])
for theta in range(len(thetaX)):
            cos_thetaX = np.cos(thetaX[theta])
            sin_thetaX = np.sin(thetaX[theta])
            rotY[theta,:,:] = np.array([[1, 0, 0], [0, cos_thetaX, sin_thetaX], \
                                        [0, -sin_thetaX, cos_thetaX]])
rotY       

NameError: name 'thetaX' is not defined

In [11]:
thetaX = np.random.randint(0,360, 10)

In [12]:
a =  ne.evaluate('sin(thetaX)') 

In [24]:
start = timeit.default_timer()
for _ in range(100000):
    np.cos(thetaX)
    np.sin(thetaX)
end = timeit.default_timer()
print(f"np trig took {end - start} seconds.")

np trig took 0.10324929199850885 seconds.


In [14]:
start = timeit.default_timer()
for _ in range(100000):
    ne.evaluate('cos(thetaX)') 
    ne.evaluate('sin(thetaX)') 
end = timeit.default_timer()
print(f"np trig took {end - start} seconds.")

np trig took 0.6204184170019289 seconds.


In [18]:
@njit
def sincos_simple(x):
    return np.sin(x), np.cos(x)

In [25]:
start = timeit.default_timer()
for _ in range(100000):
    sincos_simple(thetaX)
print(f"np trig took {end - start} seconds.")

np trig took -2.73370416599937 seconds.
